In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import seaborn as sns
import FinancialMachineLearning as fml

In [10]:
df = pd.read_parquet('./Data/IVE_tickbidask.parq')
mad = fml.madOutlier(df.price.values.reshape(-1, 1))
df = df.loc[~mad]
df

86.96
[36.17 36.25 36.21 ... 29.26 29.26 29.26]
[1.41225268 1.41537627 1.41381447 ... 1.14245268 1.14245268 1.14245268]


,price,bid,ask,size,v,dv
dates,,,,,,
2009-09-28 09:30:00,50.79,50.70,50.79,100,100,5079.00
2009-09-28 09:30:00,50.71,50.70,50.79,638,638,32352.98
2009-09-28 09:31:32,50.75,50.75,50.76,100,100,5075.00
2009-09-28 09:31:33,50.75,50.72,50.75,100,100,5075.00
2009-09-28 09:31:50,50.75,50.73,50.76,300,300,15225.00
...,...,...,...,...,...,...
2018-10-05 15:59:59,116.20,116.18,116.19,2000,2000,232400.00
2018-10-05 15:59:59,116.20,116.18,116.19,3900,3900,453180.00
2018-10-05 16:00:00,116.22,116.20,116.21,42884,42884,4983978.48


### Chapter 7. Cross-Validation in Finance

#### Exercise 1

금융에 있어서 K-Fold Cross Validation을 수행하기 전에 데이터셋을 섞는 것이 일반적으로 좋지 않은 이유는 무엇인가?데이터를 Shuffling하는 목적은 무엇인가? 데이터를 Shuffling하면 금융 데이터셋에 있어 K-Fold Cross Validation의 목적이 무의미해지는 이유는 무엇인가?

답 : 우리가 사용하는 금융 시계열 데이터셋인데, 만약 데이터셋을 섞게 된다면 순차적 시간 정보다 뒤섞이게 된다. 이는 오히려 추정기의 성능을 떨어뜨리는 결과를 낳게 된다. 일반적으로 Data Science 분야에서 교차 검증을 위해 데이터를 섞는 이유는 무작위 표본 추출로 test, validation, train set을 선택하기 위함인데, 금융 시계열 데이터에서는 Shuffling을 한다면 시간 정보가 사라지고, information leak 등의 다양한 문제가 발생하게 되어 결과적으로 목적이 무의미해진다.

#### Exercise 2

관측된 특성과 레이블로 구성된 한 쌍의 행렬$(X,y)$을 하나 구하자. 3장 연습 문제에서 도출한 데이터셋 중 하나를 사용해도 된다.

In [2]:
df = pd.read_csv('sp500featureBin.csv')
df

,Unnamed: 0,dollar,frac_diff_dollar,tW,w,bin
0,2009-11-04 14:41:43,50.4500,0.529761,0.750000,1.984399,-1.0
1,2009-11-04 15:43:57,49.8800,-0.137134,0.444444,0.077423,1.0
2,2009-11-05 09:46:35,50.2080,0.349295,0.388889,3.523273,1.0
3,2009-11-06 10:39:27,50.5448,0.038635,0.333333,3.626638,1.0
4,2009-11-06 12:37:18,50.9232,0.559177,0.333333,2.644159,1.0
...,...,...,...,...,...,...
3467,2018-09-26 10:07:41,116.5780,0.334817,0.209694,0.813016,-1.0
3468,2018-09-26 15:47:29,115.9960,0.220088,0.189087,0.495700,1.0
3469,2018-09-27 10:02:15,116.1100,0.750801,0.176905,0.055908,-1.0
3470,2018-09-27 15:39:35,115.9000,0.349971,0.166905,0.621259,1.0


In [3]:
X = df.iloc[:,:-1].values # dollar, frac_diff, tW, w
y = df.iloc[:,-1].values.reshape(-1,1) # bin

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = False, random_state = 42)

**(a)** $(X,y)$에 대해 데이터를 Shuffling하지 않고 Random Forest Classifier의 10 Fold Cross Validation의 성능을 도출해 보라

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y_train.reshape(1,-1))
weights = compute_class_weight(class_weight = 'balanced', classes = classes, y = y_train.reshape(-1,))

In [6]:
cweight = pd.DataFrame()
cweight.index = df.index
cweight['class_weight'] = np.nan
cweight.loc[df[df.bin == -1.0].index] = weights[1]
cweight.loc[df[df.bin == 1.0].index] = weights[0]
cweight

,class_weight
0,0.888158
1,1.144068
2,1.144068
3,1.144068
4,1.144068
...,...
3467,0.888158
3468,1.144068
3469,0.888158
3470,1.144068


In [7]:
#based on book recommendation
rf = RandomForestClassifier(n_estimators = 1000, criterion = "entropy", bootstrap = True,
                                n_jobs=1, random_state=42, class_weight = 'balanced_subsample', oob_score=False)
cv_gen = KFold(n_splits = 10, shuffle = False)
score = fml.cvScore(rf, X, y, sample_weight = cweight, scoring = 'neg_log_loss', cv = None, cvGen = cv_gen, pctEmbargo = 0)
print('rf_clf Mean CV score: {0:.6f}\nCV Variance: {1:.6f}'.format(score.mean(), score.var()))

rf_clf Mean CV score: -0.717943
CV Variance: 0.006818


**(b)** $(X,y)$에 대해 데이터를 섞으며 Random Forest Classifier의 10 Fold Cross Validation의 검증 성능을 도출해 보라

In [8]:
cv_gen0 = KFold(n_splits = 10, random_state = 42, shuffle = True)

score = fml.cvScore(rf, X, y, sample_weight = cweight, scoring = 'neg_log_loss', cv = None, cvGen = cv_gen0, pctEmbargo = 0)
print('rf_clf Mean CV score: {0:.6f}\nCV Variance: {1:.6f}'.format(score.mean(), score.var()))

rf_clf Mean CV score: -0.556278
CV Variance: 0.000234


**(c)** 두 결과가 많이 다른 이유는 무엇인가?

**(d)** 데이터를 Shuffling하면 Information Leak이 어떻게 일어나는가?

#### Exercise 3
Exercise 2에서 사용한 것과 동일한 $(X,y)$행렬을 사용하자

**(a)** $(X,y)$에 대해 1% Embargo를 사용한 Random Forest Classifier의 10 Fold Purged Cross Validation의 검증 성능을 도출해 보라

In [11]:
dollar_M = 1000000
dollar_df = fml.BarSampling(df, 'dv', dollar_M)
dollar_df = dollar_df.groupby(level = 0).first()
dollar_df

,price,bid,ask,size,v,dv
dates,,,,,,
2009-09-28 09:46:35,51.07,51.05,51.07,900,900,45963.00
2009-09-28 09:53:49,51.14,51.13,51.14,2000,2000,102280.00
2009-09-28 09:55:26,51.14,51.11,51.14,100,100,5114.00
2009-09-28 10:02:52,51.25,51.24,51.26,4300,4300,220375.00
2009-09-28 10:10:21,51.29,51.28,51.29,4500,4500,230805.00
...,...,...,...,...,...,...
2018-10-05 15:58:23,116.19,116.19,116.20,1354,1354,157321.26
2018-10-05 15:59:20,116.18,116.17,116.19,300,300,34854.00
2018-10-05 15:59:55,116.19,116.18,116.20,800,800,92952.00


In [9]:
rf = RandomForestClassifier(n_estimators = 1000, criterion = "entropy", bootstrap = True,
                                n_jobs = 1, random_state = 42, class_weight = 'balanced_subsample', oob_score = False)
score = fml.cvScore(rf, X, y, sample_weight = cweight, scoring = 'neg_log_loss', cv = 10, pctEmbargo = 0)
print('rf_clf Mean CV score: {0:.6f}\nCV Variance: {1:.6f}'.format(score.mean(), score.var()))

ValueError: Label Through Dates must be a pd.Series